In [31]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.exceptions import ConvergenceWarning
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Part 1
## Data Pre-Processing

In [17]:
df = pd.read_csv('sentiment_analysis.csv')
df.head()

,ID,text,label
0,7.680980e+17,Josh Jenkins is looking forward to TAB Breeder...,1
1,7.680980e+17,RT @MianUsmanJaved: Congratulations Pakistan o...,1
2,7.680980e+17,"RT @PEPalerts: This September, @YESmag is taki...",1
3,7.680980e+17,"RT @david_gaibis: Newly painted walls, thanks ...",1
4,7.680980e+17,RT @CedricFeschotte: Excited to announce: as o...,1


In [3]:
def clean_file_text(text):
    new_text = re.sub(r'http\S+', '', text)
    new_text = re.sub(r'[^\w\s]', '', new_text)
    return new_text

In [4]:
def data_preprocess(text):
  tokenizer = RegexpTokenizer(r'\w+') 
  stop_words = set(stopwords.words('english'))
  ps = PorterStemmer()
  lemmatizer = WordNetLemmatizer()
  
  file_data = clean_file_text(text)
  file_data = file_data.lower()
  file_data = clean_file_text(file_data)
  file_words = tokenizer.tokenize(file_data)
  file_words = [w for w in file_words if not w.lower() in stop_words]
  file_words = [ps.stem(w) for w in file_words]
  file_words = [lemmatizer.lemmatize(w) for w in file_words]
  return file_words

In [5]:
def corpus_count_words(text, num_words):
    word_counter = Counter()
    for words in text:
      word_counter.update(words)
    word_list = [word for (word, freq) in word_counter.most_common(num_words)]

    df_rows = []
    for words in text:
      row = [1 if word in words else 0 for word in word_list]
      df_rows.append(row)
 
    X = pd.DataFrame(df_rows, columns = word_list)
    return X

In [18]:
X = corpus_count_words(df['text'].apply(data_preprocess), 100)

In [15]:
X

,rt,love,happi,thank,amp,good,great,u,best,birthday,...,cool,heart,still,twitter,trump,made,sea,find,live,woman
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54368,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54370,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
54371,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [19]:
df['label']

0         1
1         1
2         1
3         1
4         1
         ..
550386    0
550387    0
550388    1
550389    1
550390    1
Name: label, Length: 550391, dtype: int64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.3, random_state=42)

## Logistic Refression

In [21]:
clf = LogisticRegression().fit(X_train, y_train)
# predict on train and test set
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)
# calculate train and test accuracy
train_accuracy = accuracy_score(y_train, y_train_predict)
test_accuracy = accuracy_score(y_test, y_test_predict)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
print(train_accuracy, test_accuracy)

0.8623443636071045 0.8623408713768336


With gridsearchCV and Cross-Validation

In [24]:
parameters = {'C':[0.01, 0.1, 1, 10, 100], 
              'solver':('liblinear','lbfgs', 'newton-cg'),
              'penalty':('none',  'l2')}

model = LogisticRegression()
cv = GridSearchCV(model, parameters, cv=5, n_jobs=-1)
cv.fit(X_train, y_train)

print('Best cross-validation score is :\n',cv.best_score_)
print('Best parameter:\n', cv.best_params_)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py", line 71, in _check_solver
    raise ValueError("penalty='none' is no

Best cross-validation score is :
 0.8620925910050092
Best parameter:
 {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
bestLOGmodel = LogisticRegression(C = 10, penalty= 'l2', solver = 'lbfgs')
bestLOGmodel.fit(X_train, y_train)
y_pred_LOG = bestLOGmodel.predict(X_test)
LOG_f1 = f1_score(y_test, y_pred_LOG)
LOG_acc = accuracy_score(y_test, y_pred_LOG)
print("F1 Score SVM:",LOG_f1)
print("Accuracy Score SVM:",LOG_acc)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1 Score SVM: 0.8933530419276883
Accuracy Score SVM: 0.8623590402015529


## Naive Bayes

In [35]:
clf = BernoulliNB().fit(X_train, y_train)
# predict on train and test set
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)
# calculate train and test accuracy
train_accuracy = accuracy_score(y_train, y_train_predict)
test_accuracy = accuracy_score(y_test, y_test_predict)
test_f1 = f1_score(y_test, y_test_predict)
print("F1 Score NB:",test_f1)
print("Accuracy Score NB:",test_accuracy)

F1 Score NB: 0.8907675543899827
Accuracy Score NB: 0.856720648263666


### SVM

With gridsearchCV and Cross-Validation

In [27]:
parameters = {'C':[0.001, 0.01, 0.1, 1, 10]}
model = LinearSVC(random_state=0, tol=1e-5)
cv = GridSearchCV(model, parameters, cv=5, n_jobs=-1)
cv.fit(X_train, y_train)

print('The best cross-validation score: ',cv.best_score_)
print('The best parameter:', cv.best_params_)

The best cross-validation score:  0.861236053656701
The best parameter: {'C': 0.01}


In [33]:
bestSVMmodel = LinearSVC(random_state=0, tol=1e-5, C = 0.01)
bestSVMmodel.fit(X_train, y_train)
y_pred_SVM = bestSVMmodel.predict(X_test)
SVM_f1 = f1_score(y_test, y_pred_SVM)
SVM_acc = accuracy_score(y_test, y_pred_SVM)
print("F1 Score SVM:",SVM_f1)
print("Accuracy Score SVM:",SVM_acc)


F1 Score SVM: 0.8922430876663205
Accuracy Score SVM: 0.861686793686939


### Random Forest

In [29]:
parameters = {'min_samples_split':[3, 5, 10, 15, 20],
              'n_estimators':[10, 50, 100, 200]}

model = RandomForestClassifier( random_state=0)
cv = GridSearchCV(model, parameters, cv=5, n_jobs=-1)
cv.fit(X_train, y_train)

print('The best cross-validation score: ',cv.best_score_)
print('The best parameter:', cv.best_params_)

/usr/local/lib/python3.9/dist-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


The best cross-validation score:  0.8680961270972803
The best parameter: {'min_samples_split': 15, 'n_estimators': 200}


Get accuracy and f1 socre on test set

In [32]:
bestRFmodel = RandomForestClassifier(min_samples_split = 15, n_estimators= 200, random_state=0)
bestRFmodel.fit(X_train, y_train)
y_pred_RF = bestRFmodel.predict(X_test)
RF_f1 = f1_score(y_test, y_pred_RF)
RF_acc = accuracy_score(y_test, y_pred_RF)
print("F1 Score Random Forest:",RF_f1)
print("Accuracy Score Random Forest:",RF_acc)


F1 Score Random Forest: 0.8988183986879381
Accuracy Score Random Forest: 0.8684819341319541


Result Summary for part1

In [36]:
model_list = ['Logistic Regression', 'Naive Bayes', 'SVM', 'Random Forest']
f1_list = [LOG_f1, test_f1, SVM_f1, RF_f1]
acc_list = [LOG_acc, test_accuracy,SVM_acc, RF_acc]

In [37]:
result_df = pd.DataFrame({
    'Model': model_list,
    'F1 Score': f1_list,
    'Accuracy': acc_list
})
result_df

,Model,F1 Score,Accuracy
0,Logistic Regression,0.893353,0.862359
1,Naive Bayes,0.890768,0.856721
2,SVM,0.892243,0.861687
3,Random Forest,0.898818,0.868482


Based on F1 socre, Random Forest model with min_sample_split = 15, and 200 n_estimators performed the best, this model will be used in part 2.

# Part 2